In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

In [ ]:
'''
Q1: What's the version.build_hash value?

A: '42f05b9372a9a4a470db3b52817899b99a76ee73'
'''
es_client.info()['version']['build_hash']

In [28]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
'''
Q2: What function do you use for adding your data to elastic?

A: index
'''

from tqdm.auto import tqdm

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [30]:
def es_top_hit(query):
    search_query =  {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    return response['hits']['hits'][0]

In [ ]:
'''
Q3: For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result?

A: 84.05
'''
query = 'How do I execute a command in a running docker container?'

top_hit = es_top_hit(query)
round(top_hit['_score'], 2)

In [32]:
def es_top_three(query, course_filter=None):
    search_query =  {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    if course_filter:
        search_query['query']['bool']['filter'] = course_filter

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [ ]:
'''
Q4: What's the 3rd question returned by the search engine?

A: How do I copy files from a different folder into docker container’s working directory?
'''
query = 'How do I execute a command in a running docker container?'

course_filter = { 
    "term": {
        "course": "machine-learning-zoomcamp"
    }
}

search_results = es_top_three(query, course_filter)
search_results[-1]['question']

search_results

In [34]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. 
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + f"\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
'''
Q5: What's the length of the resulting prompt?

A: 1499 (closest: 1462)
'''
query = 'How do I execute a command in a running docker container?'
prompt = build_prompt(query, search_results)
len(prompt)

In [ ]:
!pip3 install tiktoken

In [ ]:
'''
Q6: Use the encode function. How many tokens does our prompt have?

A: 331 (closest: 322)
'''
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')
prompt_to_token = encoding.encode(prompt)
tokens = len(prompt_to_token)
tokens

In [37]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

In [ ]:
'''
Bonus: Let's send the prompt to OpenAI. What's the response?

A: 
To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container using the `docker ps` command. Then, execute your desired command in the specific container. Here is how you can do it:

1. List all running containers to find the container ID:
   ```
   docker ps
   ```

2. Execute a command in the specified container:
   ```
   docker exec -it <container-id> <command>
   ```

For example, if you want to start a bash session in the running container, you would use:
```
docker exec -it <container-id> bash
```
'''
from openai import OpenAI

client = OpenAI()

query = 'How do I execute a command in a running docker container?'
course_filter = { 
    "term": {
        "course": "machine-learning-zoomcamp"
    }
}

def rag(query):
    search_results = es_top_three(query, course_filter)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

answer = rag(query)
answer

In [ ]:
'''
Bonus: How much will it cost to run 1000 requests?

A1: 150*0.005*1000 + 250*.015*1000 = $4500

A2: Q6 input tokens - 331, Q7 output tokens - 110
331*0.005*1000 + 110*.015*1000 = $3305
'''
encoding = tiktoken.encoding_for_model('gpt-4o')
response_to_token = encoding.encode(answer)
tokens = len(response_to_token)
tokens